<a href="https://colab.research.google.com/github/David4988/UniHive/blob/main/PeerHive_Sentimental_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sentimental Analysis for PeerHive**

## **1. Installing dependancies and Importing necessary packages**

In [1]:
!pip install transformers torch pandas

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/96/88/beb33a79a382fcd2aed0be5222bdc47f41e4bfe7aaa90ae1374f1d8ea2af/transformers-4.53.2-py3-none-any.whl.metadata
  Using cached transformers-4.53.2-py3-none-any.whl.metadata (40 kB)
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/c8/7b/bdcb1ed8fccb63d04bdb7635161d0ec26596d92c9d7a6cce964e7876b6c1/pandas-2.3.1-cp311-cp311-win_amd64.whl.metadata
  Using cached pandas-2.3.1-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Obtaining dependency information for huggingface-hub<1.0,>=0.30.0 from https://files.pythonhosted.org/packages/46/7b/98daa50a2db034cab6cd23a3de04fa2358cb691593d28e9130203eb7a805/huggingface_hub-0.33.4-py3-none-any.whl.metadata
  Using cached huggingface_hub-0.33.4-py3-none-any.whl.metadata (14 kB)
  Obtaining dependency information for tokenizers<0.22,>=0.21 from https://files.pythonhosted.org/packages/13/c3/cc2755ee10be859c4338c962a35b9a


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install matplotlib

  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/a6/da/948a017c3ea13fd4a97afad5fdebe2f5bbc4d28c0654510ce6fd6b06b7bd/matplotlib-3.10.3-cp311-cp311-win_amd64.whl.metadata
  Using cached matplotlib-3.10.3-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/5e/fe/4029038b4e1c4485cef18e480b0e2cd2d755448bb071eb9977caac80b77b/contourpy-1.3.2-cp311-cp311-win_amd64.whl.metadata
  Using cached contourpy-1.3.2-cp311-cp311-win_amd64.whl.metadata (5.5 kB)
Using cached matplotlib-3.10.3-cp311-cp311-win_amd64.whl (8.1 MB)
Using cached contourpy-1.3.2-cp311-cp311-win_amd64.whl (222 kB)



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from transformers import pipeline

## **2. Synthesized Dataset from GPT**

In [6]:
# 50 sample mood logs (PeerHive vibes)
data = [
    {"uid": "user001", "mood_text": "I'm feeling super anxious about exams."},
    {"uid": "user002", "mood_text": "Had a great time with friends today!"},
    {"uid": "user003", "mood_text": "Nothing special, just an average day."},
    {"uid": "user004", "mood_text": "I’m exhausted. College is draining me."},
    {"uid": "user005", "mood_text": "Feeling really hopeful about my project."},
    {"uid": "user006", "mood_text": "My motivation is totally gone..."},
    {"uid": "user007", "mood_text": "I'm proud of myself for finishing that task!"},
    {"uid": "user008", "mood_text": "Why does everything feel pointless?"},
    {"uid": "user009", "mood_text": "Super productive day, nailed every goal!"},
    {"uid": "user010", "mood_text": "I'm crying but pretending it's fine."},
    {"uid": "user011", "mood_text": "Excited to start my new internship next week!"},
    {"uid": "user012", "mood_text": "I feel like no one understands me."},
    {"uid": "user013", "mood_text": "Today’s weather makes me feel calm and peaceful."},
    {"uid": "user014", "mood_text": "Why am I so tired all the time?"},
    {"uid": "user015", "mood_text": "I got rejected again. What’s the point?"},
    {"uid": "user016", "mood_text": "Just vibing to some lo-fi beats while coding."},
    {"uid": "user017", "mood_text": "Had a breakdown but at least my room is clean now."},
    {"uid": "user018", "mood_text": "I feel overwhelmed with deadlines."},
    {"uid": "user019", "mood_text": "Finished all my tasks early. I feel amazing!"},
    {"uid": "user020", "mood_text": "Another day of pretending I’m okay."},
    {"uid": "user021", "mood_text": "Met someone new today. It was fun!"},
    {"uid": "user022", "mood_text": "Cried in the bathroom again."},
    {"uid": "user023", "mood_text": "I feel seen. Someone actually checked on me."},
    {"uid": "user024", "mood_text": "Skipped classes today. No regrets."},
    {"uid": "user025", "mood_text": "Why is everything so stressful lately?"},
    {"uid": "user026", "mood_text": "Practiced mindfulness for 5 mins. Felt nice."},
    {"uid": "user027", "mood_text": "Honestly just numb at this point."},
    {"uid": "user028", "mood_text": "Smashed my goals today! Crushing it!"},
    {"uid": "user029", "mood_text": "Nobody replied to my texts. Feel invisible."},
    {"uid": "user030", "mood_text": "Ate good food. Took a nap. 10/10 day."},
    {"uid": "user031", "mood_text": "I'm so behind on everything and panicking."},
    {"uid": "user032", "mood_text": "Late night walk helped me clear my mind."},
    {"uid": "user033", "mood_text": "Finally had the courage to speak in class."},
    {"uid": "user034", "mood_text": "I'm a failure. Nothing ever works out."},
    {"uid": "user035", "mood_text": "Studied all night. Exhausted but proud."},
    {"uid": "user036", "mood_text": "Felt really supported by my friends today."},
    {"uid": "user037", "mood_text": "Fought with my best friend. Feel awful."},
    {"uid": "user038", "mood_text": "Drank water, touched grass. Healing arc begins."},
    {"uid": "user039", "mood_text": "I don’t know what I’m doing with my life."},
    {"uid": "user040", "mood_text": "Laughing at memes got me through the day."},
    {"uid": "user041", "mood_text": "Woke up feeling peaceful for once."},
    {"uid": "user042", "mood_text": "Got ghosted again. Cool cool cool."},
    {"uid": "user043", "mood_text": "Just feeling really grateful today."},
    {"uid": "user044", "mood_text": "Burnout is hitting hard."},
    {"uid": "user045", "mood_text": "Sang my heart out in the shower. Therapeutic."},
    {"uid": "user046", "mood_text": "Back pain, brain fog, soul tired. The triple kill."},
    {"uid": "user047", "mood_text": "Had fun playing games with my roommate."},
    {"uid": "user048", "mood_text": "Panic attack in the library. Great."},
    {"uid": "user049", "mood_text": "Wrote in my journal. Helped a bit."},
    {"uid": "user050", "mood_text": "Felt like I actually belong here today."}
]

In [7]:
df = pd.DataFrame(data)
df.to_csv("mood_logs.csv", index=False)
df.head()

,uid,mood_text
0,user001,I'm feeling super anxious about exams.
1,user002,Had a great time with friends today!
2,user003,"Nothing special, just an average day."
3,user004,I’m exhausted. College is draining me.
4,user005,Feeling really hopeful about my project.


## **3. Creating the model Pipeline**

In [6]:
sentiment_model = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

Device set to use cuda:0


In [19]:
result=sentiment_model(df["mood_text"][45])

### Convert the dataframe to a list and use looping to get all rows

In [ ]:
results = sentiment_model(df["mood_text"].tolist())
label = [r["label"] for r in results]
score = [r["score"] for r in results]
df.head()

### Append those new columns to the dataframe

In [ ]:
df["sentiment_label"] = label
df["sentiment_score"] = score